In [27]:
import pandas as pd
from scipy.stats import pearsonr
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import math

In [28]:
meaning = pd.read_csv('meaning.csv')
q = dict(zip(meaning.key,meaning.question))

survey = pd.read_csv('survey.csv') 
df = survey


###### 1. GENDER DATAFRAME ######

In [29]:
GENDER_df = survey.groupby('id', as_index=False)['gender'].mean() 
#print(GENDER_df)

# 'gender' cannot be renamed as it is the title of the column in survey.csv

In [30]:
M_F = []
for row in GENDER_df['gender']:
    if row < 2 : M_F.append('Male')
    elif row < 96 : M_F.append('Female')
    elif row < 97 : M_F.append('Prefer not to answer')
    elif row < 98 : M_F.append('In another way')
    
    else: M_F.append('Not_Rated')

In [31]:
GENDER_df['M_F']= M_F 
#print(GENDER_df)

In [32]:
#GENDER_df['M_F'].value_counts()

###### 2.1 AGE RANGE #######

In [33]:
survey = survey[survey.age > 0]

In [34]:
AGE_df = survey.groupby('id', as_index=False)['age'].mean() 
#print(AGE_df)

In [35]:
# Custom Binning data https://datagy.io/pandas-cut-qcut/
# ( = dont include    [  = include in range

AGE_df['Age Group'] = pd.cut(
   AGE_df['age'], 
   [0, 18, 25, 45, 65, 75, 85, 120],
    labels=['under 18', '18-24 yrs', '25-44 yrs', '45-64 yrs', '65-74 yrs', '75-84', '85+'], 
    right= False,
)
#print(AGE_df.head())

In [36]:
#AGE_df['Age Group'].value_counts().rename_axis('Age Group').reset_index(name='counts')

###### 2.2 NUMBER OF ADULTS AND CHILDREN IN HOUSEHOLD #######

In [37]:
# ADULTS (18 yrs+)
# d7 = How many adults (aged 18+) live in your household (From meaning.csv)

ADULTS_df = pd.DataFrame()
ADULTS_df['cnt'] = df['d7'].value_counts(dropna=False)
ADULTS_df['share'] = df['d7'].value_counts(dropna=False, normalize=True).mul(100).round(1).astype(str) + '%'
ADULTS_df['adults'] = ADULTS_df.index
ADULTS_df = ADULTS_df.reset_index()
ADULTS_df = ADULTS_df.drop('index', axis=1)

#ADULTS_df

In [38]:
# CHILDREN (under 18 yrs)
# d8 = How many children aged under 18 live in your household? (From meaning.csv)

CHILD_df = pd.DataFrame()
CHILD_df['cnt'] = df['d8'].value_counts(dropna=False)
CHILD_df['% share'] = df['d8'].value_counts(dropna=False, normalize=True).mul(100).round(1).astype(str) + '%'
CHILD_df['Children (<18yrs)'] = CHILD_df.index
CHILD_df = CHILD_df.reset_index()
CHILD_df = CHILD_df.drop('index', axis=1)
#CHILD_df

In [39]:
# CHILDREN (under 18 yrs)
# d8 = How many children aged under 18 live in your household? (From meaning.csv)

c = df['d8'].value_counts(dropna=False)
p = df['d8'].value_counts(dropna=False, normalize=True).mul(100).round(1).astype(str) + '%'
#pd.concat([c,p], axis=1, keys=['counts', '%'])


##### 3. EDUCATION LEVEL ######

In [40]:
# d9 What is the highest educational level that you have achieved to date?
# EDU_df = Education level df

d9_meaning = meaning[meaning.key == 'd9']
d9_meaning['response'] = d9_meaning.response.astype('int')

EDU_df = survey.groupby('d9',as_index=True)['d9'].count()

EDU_df = survey.groupby('d9').count().reset_index()[['d9','id']]
EDU_df = EDU_df.rename(columns={'id':'cnt','d9':'response'})
EDU_df['% share'] = EDU_df.cnt / EDU_df.cnt.sum() *100

EDU_df = pd.merge(EDU_df, d9_meaning, on='response')
EDU_df = EDU_df[['response','cnt','meaning','% share']]    ##### This displays only these columns 
#EDU_df.round(1)

##### 4. HOUSE OWNERSHIP #####

In [41]:
# q7 Which of the following best describes your main home? i.e owned or rented
# HO_df = House Ownership df

q7_meaning = meaning[meaning.key == 'q7']
q7_meaning['response'] = q7_meaning.response.astype('int')

HO_df = survey.groupby('q7',as_index=True)['q7'].count()

HO_df = survey.groupby('q7').count().reset_index()[['q7','id']]
HO_df = HO_df.rename(columns={'id':'cnt','q7':'response'})
HO_df['% share'] = HO_df.cnt / HO_df.cnt.sum() *100

HO_df = pd.merge(HO_df, q7_meaning, on='response')
HO_df = HO_df[['response','cnt','meaning','% share']]    
#HO_df.round(1)


##### 5. PROPERTY TYPE #####

In [42]:
# q8 What type of property do you live in?
# PT_df = Property type df

In [43]:
q8_meaning = meaning[meaning.key == 'q8']
q8_meaning['response'] = q8_meaning.response.astype('int')

PT_df = survey.groupby('q8',as_index=True)['q8'].count()

PT_df = survey.groupby('q8').count().reset_index()[['q8','id']]
PT_df = PT_df.rename(columns={'id':'cnt','q8':'response'})
PT_df['% share'] = PT_df.cnt / PT_df.cnt.sum() *100

PT_df = pd.merge(PT_df, q8_meaning, on='response')
PT_df = PT_df[['response','cnt','meaning','% share']]    
#PT_df.round(1)

###### 6.EMPLOYMENT STATUS ######

In [44]:
# segworkingstatus What is the employment status of the Chief Income Earner in your household?The Chief Income Earner is the person with the highest income in the household. If incomes are equal
# EMP_df = Employment df

In [45]:
segworkingstatus_meaning = meaning[meaning.key == 'segworkingstatus']
segworkingstatus_meaning['response'] = segworkingstatus_meaning.response.astype('int')

EMP_df = survey.groupby('segworkingstatus',as_index=True)['segworkingstatus'].count()

EMP_df = survey.groupby('segworkingstatus').count().reset_index()[['segworkingstatus','id']]
EMP_df = EMP_df.rename(columns={'id':'cnt','segworkingstatus':'response'})
EMP_df['% share'] = EMP_df.cnt / EMP_df.cnt.sum() *100

EMP_df = pd.merge(EMP_df, segworkingstatus_meaning, on='response')
EMP_df = EMP_df[['response','cnt','meaning','% share']]    
#EMP_df.round(1)

###### 7. COMBINED ANNUAL INCOME  ######

In [46]:
# d13 What is the combined annual income of your household
# INCOME_df = income df

In [47]:
d13_meaning = meaning[meaning.key == 'd13']
d13_meaning['response'] = d13_meaning.response.astype('int')

INCOME_df = survey.groupby('d13',as_index=True)['d13'].count()

INCOME_df = survey.groupby('d13').count().reset_index()[['d13','id']]
INCOME_df = INCOME_df.rename(columns={'id':'cnt','d13':'response'})
INCOME_df['% share'] = INCOME_df.cnt / INCOME_df.cnt.sum() *100

INCOME_df = pd.merge(INCOME_df, d13_meaning, on='response')
INCOME_df = INCOME_df[['response','cnt','meaning','% share']]    
#INCOME_df.round(1)

###### 8. ELECTRIC CAR OWNERSHIP ######

In [48]:
EV_df = pd.DataFrame()
EV_df['cnt'] = df['q14_2'].value_counts(dropna=False)
EV_df['share'] = df['q14_2'].value_counts(dropna=False, normalize=True).mul(100).round(1).astype(str) + '%'
EV_df['No of EV'] = EV_df.index
EV_df = EV_df.reset_index()
EV_df = EV_df.drop('index', axis=1)

#EV_df

###### 9. HAVE ELECTRIC HEAT PUMP OVER GAS BOILER  ######

In [49]:
HP_df = pd.DataFrame()
HP_df['cnt'] = df['q14_3'].value_counts(dropna=False)
HP_df['share'] = df['q14_3'].value_counts(dropna=False, normalize=True).mul(100).round(1).astype(str) + '%'
HP_df['HP over Gas'] = HP_df.index
HP_df = HP_df.reset_index()
HP_df = HP_df.drop('index', axis=1)

#HP_df

###### 10. USE SOLAR POWERED DEVEICES ######

In [50]:
SOLAR_df = pd.DataFrame()
SOLAR_df['cnt'] = df['q14_13'].value_counts(dropna=False)
SOLAR_df['share'] = df['q14_13'].value_counts(dropna=False, normalize=True).mul(100).round(1).astype(str) + '%'
SOLAR_df['Use Solar'] = SOLAR_df.index
SOLAR_df = SOLAR_df.reset_index()
SOLAR_df = SOLAR_df.drop('index', axis=1)

#SOLAR_df

###### 11. £ SPENT ON ENERGY (ELEC/GAS) PER YEAR 

In [51]:
# q12 How much do you spend on energy per year? By that we mean electricity and/or gas.If you are not sure
# ES_df = Energy Spend dataframe

q12_meaning = meaning[meaning.key == 'q12']
q12_meaning['response'] = q12_meaning.response.astype('int')

ES_df = survey.groupby('q12',as_index=True)['q12'].count()

ES_df = survey.groupby('q12').count().reset_index()[['q12','id']]
ES_df = ES_df.rename(columns={'id':'cnt','q12':'response'})
ES_df['% share'] = ES_df.cnt / ES_df.cnt.sum() *100

ES_df = pd.merge(ES_df, q12_meaning, on='response')
ES_df = ES_df[['response','cnt','meaning','% share']]    ##### This displays only these columns 
#ES_df.round(1)



#### ETHNICITY ####

In [52]:
d5_meaning = meaning[meaning.key == 'd5']
d5_meaning['response'] = d5_meaning.response.astype('int')


ES_df = survey.groupby('d5',as_index=True)['d5'].count()
ES_df = survey.groupby('d5').count().reset_index()[['d5','id']]
ES_df = ES_df.rename(columns={'id':'cnt','d5':'response'})
ES_df['% share'] = ES_df.cnt / ES_df.cnt.sum() *100

ES_df = pd.merge(ES_df, d5_meaning, on='response')
ES_df = ES_df[['response','cnt','meaning','% share']]    ##### This displays only these columns 
#ES_df.round(1)
